In [3]:
import numpy as np
import pandas as pd
from scipy.stats import expon

# Set random seed for reproducibility
np.random.seed(42)

# Number of nodes
num_nodes = 20

# Create list of nodes
nodes = list(range(num_nodes))

# Maximum simulation time
max_time = 100

# Base intensity for the Hawkes process
base_intensity = 0.1

# Define a function to simulate the Hawkes process for each edge
def simulate_hawkes_process(base_intensity, alpha, beta, max_time):
    times = []
    current_time = 0
    intensity = base_intensity
    while current_time < max_time:
        # Sample time to next event
        inter_event_time = expon.rvs(scale=1/intensity)
        current_time += inter_event_time
        
        if current_time < max_time:
            times.append(current_time)
            # Update intensity based on the Hawkes process
            intensity = base_intensity + alpha * intensity * np.exp(-beta * inter_event_time)
            
    return times

# Randomly assign alpha and beta for each potential edge
alpha = np.random.uniform(0.05, 0.2, size=(num_nodes, num_nodes))
beta = np.random.uniform(0.5, 1.5, size=(num_nodes, num_nodes))

# Store events in a list
events = []

# Simulate the Hawkes process for each edge
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        if i != j:
            times = simulate_hawkes_process(base_intensity, alpha[i, j], beta[i, j], max_time)
            for t in times:
                events.append((i, j, t))

# Sort events by time
events.sort(key=lambda x: x[2])

# Initialize adjacency matrix
adj_matrix = np.zeros((num_nodes, num_nodes))

# Store final events with event type in a list
final_events = []

# Process events and update adjacency matrix
for event in events:
    u, v, time = event
    # Simulate random update of the adjacency matrix just before the timestamp
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adj_matrix[i, j] == 0 and np.random.rand() < 0.05:
                adj_matrix[i, j] = 1
                adj_matrix[j, i] = 1
                final_events.append((i, j, time, 0))  # Type 0 event

    # Mask off the events where A_ij = 0
    if adj_matrix[u, v] == 1:
        final_events.append((u, v, time, 1))  # Type 1 event

# Convert to DataFrame for better readability
final_events_df = pd.DataFrame(final_events, columns=['u', 'v', 'time', 'k'])

# Output the final events
final_events_df.to_csv('/Users/amberrrrrr/Desktop/huozhe/simulated_data/final_hawkes_process_events_hawkes.csv', index=False)
print("Final events have been generated and saved to 'final_hawkes_process_events_hawkes.csv'.")


Final events have been generated and saved to 'final_hawkes_process_events_hawkes.csv'.


In [4]:
final_events_df

,u,v,time,k
0,0,7,0.049522,0
1,1,17,0.049522,0
2,4,5,0.049522,0
3,4,7,0.049522,0
4,6,10,0.049522,0
...,...,...,...,...
2076,3,8,99.856937,1
2077,9,15,99.860169,1
2078,2,12,99.910555,1
2079,1,10,99.939072,1
